In [10]:
from time import time
import numpy as np
import numexpr as ne
import iarray as ia
from numba import jit

In [11]:
NSLICES = 50
SLICE_THICKNESS = 10
NTHREADS = 4
CLEVEL = 5
CLIB = ia.LZ4
BLOCKSIZE = 0
# CLEVEL = 1
# CLIB = ia.ZSTD

sexpr = "(sin(x) - 3.2) * (cos(x) + 1.2)"
sexpr2 = "(x - y) * (z - 3.) * (y - x - 2)"

In [12]:
def open_datafile(filename):
    dataset = ia.from_file(filename, load_in_mem=False)
    return dataset
t0 = time()
precipitation = open_datafile("ia-data/rea6/tot_prec/2018.iarray")
t1 = time()
print("Time to open file: %.3f" % (t1 - t0))
#print("dataset:", precipitation)

# Get a random number of slices
nt, nx, ny = precipitation.shape
shape = (NSLICES * SLICE_THICKNESS, nx, ny)
pshape = (1, nx, ny)
# tslices = np.random.choice(nt - SLICE_THICKNESS, NSLICES)

Time to open file: 0.009


In [13]:
def get_slice(dataset, i):
    return dataset[slice(tslice + i, tslice + i + 1), :, :]

def concatenate_slices(dataset):
    dtshape = ia.dtshape(shape=shape, pshape=pshape, dtype=np.float32)
    iarr = ia.empty(dtshape, clevel=CLEVEL, clib=CLIB, nthreads=NTHREADS, blocksize=BLOCKSIZE)
    for i, (_, precip_block) in enumerate(iarr.iter_write_block()):
        slice_np = ia.iarray2numpy(get_slice(dataset, i))
        precip_block[:, :, :] = slice_np
    return iarr

In [14]:
np.random.seed(1)
tslice = np.random.choice(nt - NSLICES * SLICE_THICKNESS)
t0 = time()
prec1 = concatenate_slices(precipitation)
t1 = time()
print("Time for concatenating %d slices into an ia container (1): %.3f" % (NSLICES, (t1 - t0)))
print("cratio", prec1.cratio)

Time for concatenating 50 slices into an ia container (1): 2.866
cratio 0.9999885511477474


In [15]:
np.random.seed(2)
tslice = np.random.choice(nt - NSLICES * SLICE_THICKNESS)
t0 = time()
prec2 = concatenate_slices(precipitation)
t1 = time()
print("Time for concatenating %d slices into an ia container (2): %.3f" % (NSLICES, (t1 - t0)))
print("cratio", prec2.cratio)

Time for concatenating 50 slices into an ia container (2): 2.159
cratio 0.9999885511477474


In [16]:
np.random.seed(3)
tslice = np.random.choice(nt - NSLICES * SLICE_THICKNESS)
t0 = time()
prec3 = concatenate_slices(precipitation)
t1 = time()
print("Time for concatenating %d slices into an ia container (3): %.3f" % (NSLICES, (t1 - t0)))
print("cratio", prec3.cratio)

Time for concatenating 50 slices into an ia container (3): 2.412
cratio 0.9999885511477474


In [17]:
def compute_slices(dataset):
    expr = ia.Expr(eval_flags="iterblock", clevel=CLEVEL, clib=CLIB, nthreads=NTHREADS, blocksize=BLOCKSIZE)
    expr.bind("x", dataset)
    expr.compile(sexpr)
    out = expr.eval(shape, pshape, dataset.dtype)
    return out
t0 = time()
slices_computed = compute_slices(prec1)
t1 = time()
print("Time for computing '%s' expression with 1 operand (iarray): %.3f" % (sexpr, (t1 - t0)))

Time for computing '(sin(x) - 3.2) * (cos(x) + 1.2)' expression with 1 operand (iarray): 2.114


In [18]:
def compute_slices2(dset1, dset2, dset3):
    expr = ia.Expr(eval_flags="iterblock", clevel=CLEVEL, clib=CLIB, nthreads=NTHREADS, blocksize=BLOCKSIZE)
    expr.bind("x", dset1)
    expr.bind("y", dset2)
    expr.bind("z", dset3)
    expr.compile(sexpr2)
    out = expr.eval(shape, pshape, dset1.dtype)
    return out
t0 = time()
slices_computed2 = compute_slices2(prec1, prec2, prec3)
t1 = time()
print("Time for computing '%s' expression with 3 operands (iarray): %.3f" % (sexpr2, (t1 - t0)))

Time for computing '(x - y) * (z - 3.) * (y - x - 2)' expression with 3 operands (iarray): 3.704
